## Создание словаря используемых в ВК эмодзи на основе их тональной и смысловой окраски

In [ ]:
!pip install --proxy 10.0.0.1:3128 numpy

 --upgrade pip nltk gensim pandas pymorphy2 emoji numpy

### Импорт необходимых библиотек

In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import zipfile
import numpy as np

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

## Подготовка датасета

### Чтение датасетов

#### df1

In [7]:
colnames = ['index', 'comment']

In [11]:
df1 = pd.read_csv('datasets/barneos22.csv', names=colnames, header=None)

In [12]:
df1 = df1.dropna()

In [13]:
df1

index                                            comment
1        0.0                                          Ледокол 🤣
2        1.0  Красавец! Какая же удивительная природа😊 столь...
3        2.0  [id457697599|Анастасия], да какие это люди? Ма...
4        3.0                                          Шикарный👍
5        4.0                                 "краснокнижного" )
...      ...                                                ...
1722  1721.0  [id255548341|Анастасия], люди по умнее, кому д...
1723  1722.0  [id258410293|Евгений], глупости говорите. У ме...
1724  1723.0  На видео министр здравоохранения Попов смешно ...
1725  1724.0  Огласите пожалуйста весь список, все 700 учреж...
1726  1725.0  Просматривая такие новости, у меня лично, возн...

[1662 rows x 2 columns]

#### df2

In [ ]:
df2 = pd.read_csv('datasets/barneos22.csv', names=colnames, header=None)

In [ ]:
df1 = df1.dropna()

#### df3

In [10]:
comments = comments.drop(labels = [0],axis = 0)

In [11]:
comments = comments.drop(columns = ['index'],axis = 1)

In [12]:
comments = comments.reset_index(drop=True)

In [13]:
comments

comment
0                                             Ледокол 🤣
1     Красавец! Какая же удивительная природа😊 столь...
2     [id457697599|Анастасия], да какие это люди? Ма...
3                                             Шикарный👍
4                                    "краснокнижного" )
...                                                 ...
1721  [id255548341|Анастасия], люди по умнее, кому д...
1722  [id258410293|Евгений], глупости говорите. У ме...
1723  На видео министр здравоохранения Попов смешно ...
1724  Огласите пожалуйста весь список, все 700 учреж...
1725  Просматривая такие новости, у меня лично, возн...

[1726 rows x 1 columns]

In [ ]:
comments = pd.read_csv('datasets/barneos22.csv', names=colnames, header=None)

##### Предобработка комментариев

Удаление из комментраиев-ответов имени и идентификатора пользователя [id255548341|Анастасия], которому адресован ответ 

In [14]:
pattern = r'\[(.*?)\]'

def reply_delete(doc):
    doc = re.sub(pattern, ' ', str(doc))
    return doc

In [15]:
comments['comment'] = comments['comment'].apply(reply_delete)

In [16]:
comments

comment
0                                             Ледокол 🤣
1     Красавец! Какая же удивительная природа😊 столь...
2           , да какие это люди? Маленько не дотягивают
3                                             Шикарный👍
4                                    "краснокнижного" )
...                                                 ...
1721   , люди по умнее, кому дорого свое здоровье, е...
1722   , глупости говорите. У меня не одному знакомо...
1723  На видео министр здравоохранения Попов смешно ...
1724  Огласите пожалуйста весь список, все 700 учреж...
1725  Просматривая такие новости, у меня лично, возн...

[1726 rows x 1 columns]

In [17]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    for c in str(doc):
      if emoji.is_emoji(c):
          i = doc.index(c)
          doc = doc[:i] + doc[i:]
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
        elif emoji.is_emoji(token):
            tokens.append(token)
    return tokens

In [18]:
comments['comment'] = comments['comment'].apply(lemmatize)

In [20]:
comments = comments.dropna()

In [21]:
comments

comment
0                                          [ледокол, 🤣]
1     [красавец, какой, удивительный, природа😊, стол...
2           [какой, это, человек, маленько, дотягивать]
3                                           [шикарный👍]
4                                       [краснокнижный]
...                                                 ...
1721  [человек, умный, кто, дорого, свой, здоровье, ...
1722  [глупость, говорить, у, один, знакомый, родств...
1723  [на, видео, министр, здравоохранение, попов, с...
1724   [огласить, пожалуйста, весь, список, учреждение]
1725  [просматривать, такой, новость, лично, возника...

[1726 rows x 1 columns]